In [ ]:
import pandas as pd
import os
import pyarrow as pa
import pyarrow.parquet as pq
import dask as d


sample_folder = 'new_sample_one_processed/'
merged_folder = 'merged_sample_processed/'

os.makedirs(merged_folder, exist_ok=True)

merged_dtype = files_info[0]['dtype']
base_sample = pd.read_parquet(sample_folder + files_info[0]['name'] + '.parquet', engine='pyarrow')
base_sample = base_sample.astype(merged_dtype)

print(f"Base file: {sample_folder + 'processed_' + files_info[0]['name'] + '.parquet'} is loaded")

def get_df_size_mb(df):
    return df.memory_usage(deep=True).sum() / (1024 ** 2)

def getDatatype(file):
    all_keys = {'anon_id':'object',
               'pat_enc_csn_id_coded':'Int64',
               'order_proc_id_coded':'Int64',
               'order_time_jittered_utc':'object'}
    part_keys = {
                'anon_id':'object',
               'pat_enc_csn_id_coded':'Int64',
               'order_proc_id_coded':'Int64'
    }
    for f in files_info:
        if f['name']== file:
            if len(f['merge_on']) == 4:
                all_keys.update(f['dtype'])
                return all_keys
            else:
                part_keys.update(f['dtype'])
                return part_keys


MAX_MEMORY_MB = 100
mode = 'w'
full = False

for file in files_info[1:]:
    print(f"Start merge with: {sample_folder +  'processed_'  + file['name'] + '.parquet'} ")
    df = pd.read_parquet(sample_folder + file['name'] + '.parquet' )
    dtype = getDatatype(file['name'])
    merged_dtype.update(dtype)
    df = df.astype(dtype)
    if get_df_size_mb(base_sample) < MAX_MEMORY_MB and not full: 
        base_sample = base_sample.merge(df, how = 'inner', on = file['merge_on'] , suffixes=('', '_right'))
        base_sample = base_sample.drop(columns=[col for col in base_sample.columns if col.endswith('_right')])
        base_sample.to_parquet(merged_folder + 'merged_ARMD.parquet' , engine='pyarrow', index=False)
    else: 
        full = True
        base_sample = pd.DataFrame()
         
    if full: 
        parquet_file = pq.ParquetFile(merged_folder + 'merged_ARMD.parquet')
        for i in range(parquet_file.num_row_groups):
            chunk = parquet_file.read_row_group(i).to_pandas()
            temp = chunk.merge(df, how = 'inner', on = file['merge_on'], suffixes=('', '_right') )
            temp = temp.drop(columns=[col for col in temp.columns if col.endswith('_right')])
            temp = temp.astype(merged_dtype)
            parquet_path = merged_folder + 'temp_merged_ARMD.parquet'
            table = pa.Table.from_pandas(temp)
            if os.path.exists(parquet_path) and mode == 'a':
                with pq.ParquetWriter(parquet_path, table.schema, compression='snappy', use_dictionary=True) as writer:
                    writer.write_table(table)
            else:
                pq.write_table(table, parquet_path, compression='snappy', use_dictionary=True)
                mode = 'a'
                
        mode = 'w'
        
        for i in range(parquet_file.num_row_groups):
            chunk = parquet_file.read_row_group(i).to_pandas() 
            chunk = chunk.astype(merged_dtype)
            parquet_path = merged_folder + 'temp_merged_ARMD.parquet'
            table = pa.Table.from_pandas(chunk)
            if os.path.exists(parquet_path) and mode == 'a':
                with pq.ParquetWriter(parquet_path, table.schema, compression='snappy', use_dictionary=True) as writer:
                    writer.write_table(table)
            else:
                pq.write_table(table, parquet_path, compression='snappy', use_dictionary=True)
                mode = 'a'
                   
print("The merge is done Alhamdulillah :) ")          